# Exploratory Data Analysis (EDA) - Heart Disease Risk Dataset

This notebook contains a comprehensive exploration of the BRFSS dataset used for heart disease risk prediction.

## Objectives:
1. Understand the dataset structure and content
2. Analyze missing values patterns
3. Extract and link feature labels from HTML codebook
4. Explore feature correlations with target variable
5. Identify relevant feature groups for modeling


## 1. Import Libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bs4
import re

# Configure plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


## 2. Load Dataset


In [ ]:
# Load training data
df = pd.read_csv("data/train.csv")

# Basic information
print(f"Dataset shape: {df.shape}")
print(f"Number of features: {df.shape[1]}")
print(f"Number of samples: {df.shape[0]}")
print(f"\nTarget variable: {df.columns[-1]}")
print(f"\nTarget distribution:")
print(df['TARGET'].value_counts())
print(f"\nClass balance: {df['TARGET'].value_counts(normalize=True)}")


## 3. Extract Feature Labels from HTML Codebook

The dataset comes with an HTML codebook containing detailed descriptions of each feature. We parse this to understand what each column represents.


In [ ]:
html_file = "data/USCODE22_LLCP_102523.HTML"

# Read and parse HTML
with open(html_file, encoding="latin-1") as f:
    html = f.read()

soup = bs4.BeautifulSoup(html, "html.parser")

# Extract question information
questions = []

for td in soup.find_all("td", class_="l m linecontent"):
    text = td.get_text(separator=" ", strip=True)
    text = text.replace("\xa0", " ")  
    text = re.sub(r"\s+", " ", text)

    # Extract different fields using regex
    label_match = re.search(r"Label:\s(.*?)\s+(?:Section Name:|Core Section Name:)", text, re.IGNORECASE)
    section_name_match = re.search(r"(?:Section Name:|Core Section Name:)\s(.*?)\s+(?:Section Number:|Core Section Number:|Module Number:)", text, re.IGNORECASE)
    section_number_match = re.search(r"(?:Section Number:|Core Section Number:|Module Number:)\s*([0-9A-Za-z]+)", text, re.IGNORECASE)
    sas_match = re.search(r"SAS Variable Name:\s*([A-Za-z0-9_]+)", text, re.IGNORECASE)

    questions.append({
        "Label": label_match.group(1).strip() if label_match else "",
        "Section Name": section_name_match.group(1).strip() if section_name_match else "",
        "Section Number": section_number_match.group(1).strip() if section_number_match else "", 
        "SAS_Variable_Name": sas_match.group(1).strip() if sas_match else ""
    })

# Create DataFrame
labels_df = pd.DataFrame(questions)

# Save for later use
labels_df.to_csv("data/labels_questions.csv", index=False, encoding="utf-8")

print(f"Extracted {len(labels_df)} feature descriptions")
print(f"\nNumber of unique sections: {labels_df['Section Name'].nunique()}")
print(f"\nSample of extracted data:")
labels_df.head(10)


## 4. Missing Values Analysis

Understanding patterns in missing data is crucial for preprocessing decisions.


In [ ]:
# Calculate missing value percentages
missing_percent = df.isna().mean() * 100
missing_percent = missing_percent.sort_values(ascending=False)

# Display statistics
print("Missing Values Statistics:")
print(f"Columns with 100% missing: {(missing_percent == 100).sum()}")
print(f"Columns with >90% missing: {(missing_percent > 90).sum()}")
print(f"Columns with >50% missing: {(missing_percent > 50).sum()}")
print(f"Columns with 0% missing: {(missing_percent == 0).sum()}")

# Show top features with missing values
print(f"\nTop 20 features with most missing values:")
missing_percent.head(20)


In [ ]:
# Visualize distribution of missing values
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.hist(missing_percent, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Percentage of Missing Values (%)')
plt.ylabel('Number of Features')
plt.title('Distribution of Missing Values Across Features')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
# Group by ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
categories = pd.cut(missing_percent, bins=bins, labels=labels, include_lowest=True)
category_counts = categories.value_counts().sort_index()

plt.bar(range(len(category_counts)), category_counts.values, edgecolor='black', alpha=0.7)
plt.xticks(range(len(category_counts)), category_counts.index, rotation=45, ha='right')
plt.xlabel('Missing Value Range')
plt.ylabel('Number of Features')
plt.title('Features Grouped by Missing Value Percentage')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Feature Correlation with Target

Identify which features are most correlated with the target variable.


In [ ]:
# Convert to numeric for correlation
df_num = df.apply(pd.to_numeric, errors='coerce')

# Calculate correlation with target
target_col = 'TARGET'
corr_with_target = df_num.corr(numeric_only=True)[target_col].drop(labels=[target_col], errors='ignore')

# Sort by absolute correlation
corr_sorted = corr_with_target.abs().sort_values(ascending=False)

print(f"Features with |correlation| > 0.1: {(corr_sorted > 0.1).sum()}")
print(f"\nTop 20 features correlated with target:")
print(pd.DataFrame({
    'Feature': corr_sorted.head(20).index,
    'Correlation': corr_with_target[corr_sorted.head(20).index].values,
    'Abs Correlation': corr_sorted.head(20).values
}))


In [ ]:
# Visualize correlation with target
plt.figure(figsize=(12, 8))
top_n = 25
top_features = corr_sorted.head(top_n).index
top_corr = corr_with_target[top_features]

colors = ['red' if x < 0 else 'green' for x in top_corr.values]
plt.barh(range(len(top_corr)), top_corr.values, color=colors, alpha=0.7, edgecolor='black')
plt.yticks(range(len(top_corr)), top_features)
plt.xlabel('Correlation with Target')
plt.title(f'Top {top_n} Features Correlated with Heart Disease Risk')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


## 6. Analysis by Feature Section

Group features by their section to understand which categories are most informative.


In [ ]:
# Remove columns that are 100% NaN for analysis
cols_all_nan = df.columns[df.isna().all()].tolist()
df_clean = df.drop(cols_all_nan, axis=1)
labels_clean = labels_df[~labels_df['SAS_Variable_Name'].isin(cols_all_nan)].reset_index(drop=True)

print(f"Removed {len(cols_all_nan)} columns with 100% NaN")
print(f"Remaining features: {df_clean.shape[1]}")


In [ ]:
# Compute NaN percentage for each feature
nan_percent = df_clean.isna().mean() * 100
nan_percent = nan_percent.reset_index()
nan_percent.columns = ["SAS_Variable_Name", "NaN_percent"]

# Compute correlation for each feature
corrs = []
for col in df_clean.columns:
    if col == target_col:
        continue
    try:
        if pd.api.types.is_numeric_dtype(df_clean[col]):
            corr = df_clean[col].corr(df_clean[target_col])
            corrs.append((col, corr))
        else:
            corrs.append((col, np.nan))
    except Exception:
        corrs.append((col, np.nan))

corr_df = pd.DataFrame(corrs, columns=["SAS_Variable_Name", "corr_with_target"])

# Merge with labels
merged = (
    nan_percent
    .merge(corr_df, on="SAS_Variable_Name", how="left")
    .merge(labels_clean[["SAS_Variable_Name", "Section Name"]],
           on="SAS_Variable_Name", how="left")
)

# Aggregate by section
section_summary = merged.groupby("Section Name").agg(
    mean_nan_percent=("NaN_percent", "mean"),
    mean_abs_corr=("corr_with_target", lambda x: np.nanmean(np.abs(x))),
    num_features=("SAS_Variable_Name", "count")
).sort_values("mean_abs_corr", ascending=False)

print("Section Summary (sorted by mean absolute correlation):")
print(section_summary.head(15))


In [ ]:
# Visualize section importance
fig, ax1 = plt.subplots(figsize=(14, 8))

# Sort by correlation for better visualization
section_summary_sorted = section_summary.sort_values("mean_abs_corr")

# Bar plot for mean NaN percentage
ax1.barh(range(len(section_summary_sorted)), 
         section_summary_sorted['mean_nan_percent'], 
         color="skyblue", alpha=0.7, label="Mean % NaN")
ax1.set_xlabel("Average Missing Values (%)", color="skyblue")
ax1.set_ylabel("Section Name")
ax1.set_yticks(range(len(section_summary_sorted)))
ax1.set_yticklabels(section_summary_sorted.index, fontsize=8)
ax1.tick_params(axis='x', labelcolor="skyblue")

# Overlay line plot for correlation
ax2 = ax1.twiny()
ax2.plot(section_summary_sorted['mean_abs_corr'], 
         range(len(section_summary_sorted)),
         color="darkred", marker="o", linewidth=2, markersize=5, label="Mean |corr|")
ax2.set_xlabel("Mean Absolute Correlation with Target", color="darkred")
ax2.tick_params(axis='x', labelcolor="darkred")

plt.title("Feature Sections: Missing Values vs. Correlation with Target", pad=20)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


## 7. Correlation Matrix of Selected Features

Examine correlations between the most relevant features (|correlation| > 0.1 with target).


In [ ]:
# Select features with high correlation and low missing values
high_corr_features = corr_with_target[abs(corr_with_target) > 0.1].index.tolist()
nan_pct_dict = nan_percent.set_index("SAS_Variable_Name")["NaN_percent"].to_dict()

# Filter features with < 70% missing values
selected_for_viz = [f for f in high_corr_features if nan_pct_dict.get(f, 100) < 70]

print(f"Selected {len(selected_for_viz)} features for correlation matrix visualization")
print(f"(|correlation| > 0.1 and < 70% missing values)")

# Compute correlation matrix
if len(selected_for_viz) > 1:
    corr_matrix = df_clean[selected_for_viz + [target_col]].corr()
    
    # Visualize
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
    sns.heatmap(corr_matrix, mask=mask, cmap="vlag", center=0, 
                vmin=-1, vmax=1, square=True, 
                linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title("Correlation Matrix of Selected Features")
    plt.tight_layout()
    plt.show()


## 8. Key Insights

### Dataset Characteristics:
1. **Imbalanced Dataset**: Only ~9% of samples have positive heart disease risk
2. **High Dimensionality**: 325 features with varying levels of missing data
3. **Missing Data Patterns**: Many features have >90% missing values (likely optional questions)

### Most Informative Feature Categories:
- **Demographics** (Age variables): Strongest correlation with target
- **Health Status**: General health perception
- **Chronic Conditions**: Diabetes, arthritis
- **Lifestyle Factors**: Smoking, exercise

### Recommendations for Modeling:
1. Focus on features with |correlation| > 0.1 (23 features)
2. Remove columns with 100% missing values
3. Handle class imbalance with class weighting or resampling
4. Use imputation strategies for remaining missing values
5. Consider feature engineering from highly correlated groups
